# Pill Working Example
https://arxiv.org/abs/2012.02904

The justification is that `(onDay pill Wednesday) (beforeTime pill afternoon)`. 

In [1]:
import commonsense.conceptnet as kb
from commonsense.logical_classes import Fact, to_data_frame, parse_file_to_fact_list, parse_file_to_event_list, parse_raw_file_to_event_list, create_facts_from_file
from monitor.reasonableness_monitor import SnapshotMonitor

In [2]:
events = [Fact('Levodopa', 'onDate', 'Wednesday')]
preference = Fact('pill', 'before', 'activity')

cn = kb.ConceptNet()
pill_monitor = SnapshotMonitor()

additional_facts = create_facts_from_file("pill_facts.txt")  # additional information
facts=pill_monitor.explain(events, additional_facts, preference, verbose=True)

print(facts)

Preference is pill before activity
chain is nothing adding more
has a chain, currently is [Fact(subject='pill', predicate=' before', object='appt', reason='Rule fired', score=1.0, count=1)]
Explanation is [Fact(subject='pill', predicate=' before', object='appt', reason='Rule fired', score=1.0, count=1), Fact(subject='appt', predicate=' IsA', object='activity', reason='Given', score=1.0, count=1)]
(pill  before appt), (appt  IsA activity), (appt  atTime 1pm), (appt  onDay Wednesday)


# Hinge Example

In [3]:
events = [Fact('leg', 'attach', 'top')]
preference = Fact('screw', 'before', 'leg')

cn = kb.ConceptNet()
pill_monitor = SnapshotMonitor()

additional_facts = create_facts_from_file("furniture_facts.txt")  # additional information
facts=pill_monitor.explain(events, additional_facts, preference=preference,  verbose=True, kb_query=False)

print(facts)

Preference is screw before leg
chain is nothing adding more
has a chain, currently is [Fact(subject='screw', predicate=' into', object='leg', reason='Given', score=1.0, count=1)]
has a chain, currently is [Fact(subject='screw', predicate=' into', object='leg', reason='Given', score=1.0, count=1), Fact(subject='leg', predicate='attach', object='top', reason=None, score=1.0, count=1)]
has a chain, currently is [Fact(subject='screw', predicate=' into', object='leg', reason='Given', score=1.0, count=1), Fact(subject='leg', predicate='attach', object='top', reason=None, score=1.0, count=1), Fact(subject='top', predicate=' flip', object='top1', reason='Given', score=1.0, count=1)]
Explanation is [Fact(subject='screw', predicate=' into', object='leg', reason='Given', score=1.0, count=1), Fact(subject='leg', predicate='attach', object='top', reason=None, score=1.0, count=1), Fact(subject='top', predicate=' flip', object='top1', reason='Given', score=1.0, count=1), Fact(subject='top1', predicat

# Testing Analogy

# Explaining B above D
We are using analogy to infer that B is above D because we know that A is on B, B is on C, and C on D.  

In [4]:
def parse(program):
    "Read a Scheme expression from a string."
    return read_from_tokens(tokenize(program))

def tokenize(s):
    "Convert a string into a list of tokens."
    return s.replace('(',' ( ').replace(')',' ) ').split()

def read_from_tokens(tokens):
    "Read an expression from a sequence of tokens."
    if len(tokens) == 0:
        raise SyntaxError('unexpected EOF while reading')
    token = tokens.pop(0)
    if '(' == token:
        L = []
        while tokens[0] != ')':
            L.append(read_from_tokens(tokens))
        tokens.pop(0) # pop off ')'
        return L
    elif ')' == token:
        raise SyntaxError('unexpected )')
    else:
        return atom(token)

def atom(token):
    "Numbers become numbers; every other token is a symbol."
    try: 
        temp = int(token)
    except ValueError:
        try: 
            temp = float(token)
        except ValueError:
            return Symbol(token)

In [5]:
def read_relations(): 
    file1 = open('datasets/PAX/analogy/sme_relations_b_above_d.txt', 'r')
    count = 0
    lines = []
    continue_phrase = False

    while True:
        count += 1

        # Get next line from file
        line = file1.readline()

        # if line is empty
        # end of file is reached
        if not line:
            break
        elif line.strip()=="" or line.startswith(";;"):
            continue
        else:
            lines.append(line.strip())
    print("Found %d usable lines and facts"%len(lines))
    return lines

my_facts = read_relations()

def explain_analogy(query: str, facts):
    for sentence in facts:
        if query in sentence:
            print(sentence)
            print(parse(sentence))

explain_analogy('(isAbove B D)', my_facts)

Found 40 usable lines and facts
(candidateInferenceContent (CiFn 0 7 (MatcherFn 18 0)) (subtask (isAbove B D) (isOn C D)))


NameError: name 'Symbol' is not defined

# Small examples for Demo


In [ ]:
from reasoner.production import IF, AND, OR, NOT, THEN, DELETE, forward_chain, pretty_goal_tree, match, populate, simplify
from reasoner.data import *
import pprint


pp = pprint.PrettyPrinter(indent=1)
pprint = pp.pprint

In [ ]:
query = Fact('penguin', 'IsA', 'animal')

cn = kb.ConceptNet()
monitor = SnapshotMonitor()

# Still needs to be translated to facts. 
transitive_rule = IF(AND('(?x) IsA (?y)', '(?y) IsA (?z)'),
                     THEN('(?x) IsA (?z)'))
facts = monitor.explain_fact(query)
data_facts = [fact.to_string() for fact in facts]
print("started with %s facts"%len(data_facts))

chained_data = forward_chain([transitive_rule], data_facts)

print("ended with %s facts"%len(chained_data))

In [ ]:
def backchain_to_goal_tree(rules, hypothesis):
    """
    Takes a hypothesis (string) and a list of rules (list
    of IF objects), returning an AND/OR tree representing the
    backchain of possible statements we may need to test
    to determine if this hypothesis is reachable or not.

    This method should return an AND/OR tree, that is, an
    AND or OR object, whose constituents are the subgoals that
    need to be tested. The leaves of this tree should be strings
    (possibly with unbound variables), *not* AND or OR objects.
    Make sure to use simplify(...) to flatten trees where appropriate.
    """
    top_level = [hypothesis]
    
    for rule in rules:
        matching = match(rule.consequent(), hypothesis)
    
        if matching is not None:
            tree = []
            next = populate(rule.antecedent(), matching)
            ant_stuff = type(rule.antecedent())
    
            if ant_stuff is str:  # leaf
                tree.append(backchain_to_goal_tree(rules, next))
            else:  # it's a phrase
                for hyp in next:
                    tree.append(backchain_to_goal_tree(rules, hyp))
    
            if ant_stuff == OR:
                top_level.append(OR(tree))
            else:
                top_level.append(AND(tree))
    return simplify(OR(top_level))

# pretty_goal_tree(backchain_to_goal_tree(zookeeper_rules, 'opus is a penguin'))
# result = backchain_to_goal_tree(zookeeper_rules, 'opus is a penguin')

#pretty_goal_tree(backchain_to_goal_tree([transitive_rule], 'penguin IsA animal'))

# Lego example


In [ ]:
from commonsense.logical_classes import preprocess

events = preprocess()  # This reads in the file: allnewsemanticdatawithmultimodal0025.txt by default
cn = kb.ConceptNet()
cn_monitor = SnapshotMonitor()

cn_monitor.explain(events, additional_facts)

# Older Lego Example

In [ ]:
import commonsense.conceptnet as kb
from commonsense.logical_classes import to_data_frame, parse_file_to_fact_list, parse_file_to_event_list, parse_raw_file_to_event_list, create_facts_from_file
from monitor.reasonableness_monitor import SnapshotMonitor

cn = kb.ConceptNet()  # choose your KB 
cn_monitor = SnapshotMonitor()  # labels=labels, data=data, rules=rules)
print("Made a ConceptNet monitor, now trying to explain")

events = parse_file_to_event_list("datasets/PAX/output_feb25.txt")  # This reads in the file. 
additional_facts = create_facts_from_file("gaze_facts.txt") # should call rules 
cn_monitor.explain_events(events, additional_facts)


In [ ]:
facts

f = to_data_frame(facts)
events = f.groupby("subject")
f.groupby("subject").count()
events.first()
events.nth(0)

facts